In [9]:
parent_dir = "C:/Users/Davide/Il mio Drive/Universita/Dottorato/git/EnergyCommunity.jl/run_cloud"

n_iter = 2
EC_size_list_enum = [3,4] #[5, 10, 20]  # List of sizes of the EC to test in enum mode

2-element Vector{Int64}:
 3
 4

In [1]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML

In [7]:
list_enums = Dict(
    size_enum=>load("$parent_dir/results_paper/enum/enum_simulations_results_$size_enum.jld2")
    for size_enum in EC_size_list_enum
)

ArgumentError: ArgumentError: No file exists at given path: C:/Users/Davide/Il mio Drive/Universita/Dottorato/git/EnergyCommunity.jl/run_cloud/results_paper/enum/enum_simulations_results_3.jld2

### Create reward redistribution of enum modes

In [ ]:
df_reward_enum = DataFrame()

for (id_enum, enum_data) in enumerate(list_enums)
    if nrow(df_reward) == 0
        df_reward_enum = enum_data["df_reward_enum"]
    else
        df_reward_enum = innerjoin(df_reward_enum, enum_data["df_reward_enum"], on=:user_set)
    end
end

# sort by user set
sort!(df_reward_enum, :user_set)

df_reward_enum

### Create comparison of computational time

In [ ]:
df_time_enum = DataFrame()

for (id_enum, enum_data) in enumerate(list_enums)
    df_time_temp = DataFrame(enum_data["dict_time_enum"])]./3600

    df_time_temp[!, "EC size"] = [L"Time [h]"]
    df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, ["EC size"; "title"; names(df_time_temp)[1:end-2]]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

In [11]:
list_iter = Dict(
    id_run=>load("$parent_dir/results_paper/iter/iter_simulations_results_$id_run.jld2")
    for id_run in 1:n_iter
)